### 일급 객체
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

In [1]:
# 파이썬에서 함수는 콘솔 섹션에서 (즉 런타임에서) 만들 수 있다.
def factorial(n) :
    """return n!"""
    return 1 if n < 2 else n * factorial(n-1)

print(factorial(42))
# help(func) 에서 나오는 text는 __doc__ 에서 가져온 것
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
return n!
<class 'function'>


In [2]:
fact = factorial
fact

<function __main__.factorial(n)>

In [3]:
fact(5)

120

In [4]:
map(factorial, range(11))

In [5]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

### 고위함수 (higher-order function)
- 함수를 인수로 받거나, 결과로 반환하는 함수

In [6]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key = len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [7]:
# key 에는 인수를 하나 받는 함수는 모두 들어갈 수 있다.
def reverse(word) :
    return word[::-1]

sorted(fruits, key = reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [8]:
# map, filter 보다는 지능형 리스트를 이용하자
print(list(map(fact, range(6))))
print([fact(n) for n in range(6)])

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]


In [9]:
print(list(map(fact, filter(lambda n : n % 2, range(6)))))
print([fact(n) for n in range(6) if n % 2])

[1, 6, 120]
[1, 6, 120]


In [10]:
# 고위 함수의 인수로 제공하는 경우 외에는 람다 함수는 잘 쓰이지 않는다.
# 구현이 까다롭고 가독성도 떨어지기 때문이다.
sorted(fruits, key = lambda x : x[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## 사용자 정의 callable

In [11]:
import random

class BingoCage :
    
    def __init__(self, items) :
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self) :
        try :
            return self._items.pop()
        except :
            raise LookupError("pick from empty BingoCage")
        
    def __call__(self) :
        return self.pick()

In [12]:
bingo = BingoCage(range(6))

# 같은 역할을 하는 두 method
print(bingo.pick())
print(bingo())
print(callable(bingo))

0
5
True


## 함수 인트로스펙션

In [13]:
dir(factorial)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
# 일반 객체에는 존재하지 않는 함수 속성
class C : pass
obj = C()
def func() : pass
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__builtins__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## 위치 매개변수, 키워드 전용 매개변수

In [15]:
def tag(name, *content, cls = None, **attrs) :
    """하나 이상의 HTML 태그를 생성한다."""
    if cls is not None :
        attrs['class'] = cls
    if attrs :
        attr_str = ''.join(f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    else :
        attr_str = ''
        
    if content :
        return '\n'.join(f"<{name}{attr_str}>{c}</{name}>" for c in content)
    else :
        return f"<{name}{attr_str} />"

In [16]:
tag('br')

'<br />'

In [17]:
tag('p', 'hello')

'<p>hello</p>'

In [18]:
tag('p', 'hello', 'world')

'<p>hello</p>\n<p>world</p>'

In [19]:
tag('p','hello',id=33)

'<p id="33">hello</p>'

In [20]:
print(tag('p', 'hello', 'world', cls = 'sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [21]:
tag(content='testing', name = 'img')

'<img content="testing" />'

In [22]:
my_tag = {'name' : 'img', 'title' : 'Sunset Boulevard', 'src' : 'Sunset.jpg', 'cls' : 'framed'}
tag(**my_tag)

'<img class="framed" src="Sunset.jpg" title="Sunset Boulevard" />'

#### 주의사항
- 위치 매개변수가 먼저 전달된 다음 키워드 매개변수가 전달됨

In [23]:
# 위치 매개변수가 먼저 전달되었기 때문에 name 에 'p'를 할당 할 수 없어 에러발생
tag('img', name = 'p')

TypeError: tag() got multiple values for argument 'name'

## 매개변수에 대한 정보 읽기

In [24]:
from clip import clip
from inspect import signature

sig = signature(clip)
str(sig)

'(text, max_len=80)'

In [25]:
# POSITIONAL_OR_KEYWORD : 위치 인수나 키워드 인수로 전달할 수 있는 매개변수 
for name, param in sig.parameters.items() :
    print(f"{param.kind} : {name} = {param.default}")

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [26]:
# signature 가 제공하는 bind 메소드를 통해 어떻게 매개변수가 바인딩 되었는지, 오류가 없는지 확인할 수 있다.
sig = signature(tag)
my_tag = {'name' : 'img', 'title' : 'Sunset Boulevard', 'src' : 'sunset.jpg',
          'cls' : 'framed'}
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [27]:
for name, value in bound_args.arguments.items() :
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [28]:
# 함수가 바인딩 되지 않으면 오류가 발생한다.
del my_tag['name']
bound_args = sig.bind(**my_tag)


TypeError: missing a required argument: 'name'

## 함수 애너테이션

In [29]:
# 함수 선언에 '-> str' 형태로 애너테이션을 추가할 수 있다. (swift를 사용해봤다면 익숙한 문법)
# 클래스, 문자열이 저장되며 함수에 전혀 영향을 미치지 않는다. 단지 __annotations__에 저장될 뿐이다.
from clip_annat import clip
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [35]:
sig = signature(clip)
sig.return_annotation

str

In [47]:
for param in sig.parameters.values() :
    print(f"{str(param.annotation) : <13} : {param.name} = {param.default}")

<class 'str'> : text = <class 'inspect._empty'>
int > 0       : max_len = 80


## 함수형 프로그래밍을 위한 패키지

In [49]:
# reduce는 누적집계하는 경우 사용된다.
from functools import reduce

def fact(n) :
    return reduce(lambda a,b : a*b, range(1,n+1))
fact(10)

3628800

In [50]:
from operator import mul

def func(n) :
    return reduce(mul, range(1,n+1))
fact(10)

3628800

In [57]:
# itemgetter()에 여러개의 인덱스를 전달하여 객체를 생성한다.
# 해당 객체에 시퀀스를 매개변수로 call 하면 저장된 인덱스값들로 구성된 튜플을 반환한다.
from operator import itemgetter
itemget = itemgetter(1,3)
lst = [1,2,3,4]
itemget(lst)


(2, 4)

In [59]:
metro_data = [
    ("Tokyo", "JP", 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', "IN", 21.935, (28.613889, 77.208889)),
    ('Mexico City', "MX", 20.142, (19.433333, -99.133333)),
    ('New York-Newark', "US", 20.1-4, (40.808611, -74.020386)),
    ("Sao Paulo", "BR", 19.649, (-23.547778, -46.635833))
]

for city in sorted(metro_data, key = itemgetter(1)) :
    print(city)
    
# 이는 다음과 동일하다.
print()
for city in sorted(metro_data, key = lambda x : x[1]) :
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 16.1, (40.808611, -74.020386))

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 16.1, (40.808611, -74.020386))


In [72]:
from collections import namedtuple

LatLong = namedtuple ("LatLong", "lat long")
Metropolis = namedtuple("Metropolis", "name cc pop coord")

metro_areas = [Metropolis(name, cc , pop, LatLong(lat, long)) 
               for name, cc, pop, (lat, long) in metro_data]
metro_areas

[Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667)),
 Metropolis(name='Delhi NCR', cc='IN', pop=21.935, coord=LatLong(lat=28.613889, long=77.208889)),
 Metropolis(name='Mexico City', cc='MX', pop=20.142, coord=LatLong(lat=19.433333, long=-99.133333)),
 Metropolis(name='New York-Newark', cc='US', pop=16.1, coord=LatLong(lat=40.808611, long=-74.020386)),
 Metropolis(name='Sao Paulo', cc='BR', pop=19.649, coord=LatLong(lat=-23.547778, long=-46.635833))]

In [74]:
metro_areas[0].coord.lat

35.689722

In [75]:
# itemgetter 에 인덱스 대신 key 가 들어갔다고 보면 된다.
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key = attrgetter('coord.lat')) :
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [78]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [85]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
print(s.upper())

THE TIME HAS COME
THE TIME HAS COME


In [86]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

In [91]:
# 2개 이상의 함수를 인수로 받는 함수의 인수를 고정시켜 새로운 함수를 만든다.
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [92]:
lst = ['abc', 'cad', 'gfa']
reverse_sort = partial(sorted, key = lambda x : x[::-1])
reverse_sort(lst)

['gfa', 'abc', 'cad']